In [1]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 29.08 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [5]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else :
      data[key]=[row[1]]
    

In [6]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')


FileExistsError: ignored

In [9]:
os.mkdir('master_data/testing')

In [11]:
for emotion in emotion_list:
  os.mkdir(os.path.join('/content/master_data/training/',emotion))
  os.mkdir(os.path.join('/content/master_data/testing/',emotion))

In [12]:
from shutil import copyfile
split_size=0.8
for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source=os.path.join('/content/facial_expressions/images',image)
    dest=os.path.join('/content/master_data/testing',emotion,image)
    copyfile(source,dest)

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten


In [40]:
model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation="relu",input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu",input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024,activation="relu"),
    Dense(8,activation="softmax") 
])
model.compile(optimizer=Adam(lr=0.01),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 6400)             

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [36]:
test_dir='/content/master_data/testing'
train_dir='/content/master_data/training'
train_datagen=ImageDataGenerator(rescale=1/255.0)
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode="categorical",
    batch_size=128
)
test_datagen=ImageDataGenerator(rescale=1/255.0)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode="categorical",
    batch_size=128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [38]:
es=EarlyStopping(monitor='val_accuracy',patience=2)

In [39]:
model=model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
86/86 [==============================] - 20s 223ms/step - loss: 1.2387 - accuracy: 0.4810 - val_loss: 1.0350 - val_accuracy: 0.5000
Epoch 2/10
86/86 [==============================] - 19s 217ms/step - loss: 0.9889 - accuracy: 0.5714 - val_loss: 0.9639 - val_accuracy: 0.6207
Epoch 3/10
86/86 [==============================] - 19s 218ms/step - loss: 0.8351 - accuracy: 0.7062 - val_loss: 0.9286 - val_accuracy: 0.6408
Epoch 4/10
86/86 [==============================] - 19s 222ms/step - loss: 0.7575 - accuracy: 0.7380 - val_loss: 0.9429 - val_accuracy: 0.6371
Epoch 5/10
86/86 [==============================] - 19s 221ms/step - loss: 0.7249 - accuracy: 0.7461 - val_loss: 1.0069 - val_accuracy: 0.6565
Epoch 6/10
86/86 [==============================] - 19s 222ms/step - loss: 0.6730 - accuracy: 0.7684 - val_loss: 1.0031 - val_accuracy: 0.6513
Epoch 7/10
86/86 [==============================] - 19s 220ms/step - loss: 0.6239 - accuracy: 0.7847 - val_loss: 0.9406 - val_accuracy: 0.6652